In [ ]:

!pip install accelerate==0.26.0
!pip install datasets
!pip install torch
!pip install evaluate
import torch
import evaluate
import sklearn.metrics
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, AutoConfig
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import Trainer
import numpy as np
import pandas as pd
import datasets
from sklearn.model_selection import train_test_split

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/huggingface_cli.py", line 49, in main
    servic

First read the file.

In [ ]:
dataset = load_dataset('financial_phrasebank', 'sentences_50agree')
data_train, data_val = train_test_split(dataset["train"], test_size=0.2, random_state=11)
goldstandard=data_val["label"]
with open("/content/drive/MyDrive/ipynb/final_project/goldlabel_financial_phrasebank.txt", 'w', encoding='utf-8') as f:
  for i in range(len(goldstandard)):
    f.write(str(goldstandard[i]))
    f.write('\n')
data_train = datasets.Dataset.from_dict(data_train)
data_val = datasets.Dataset.from_dict(data_val)


Tokenize the text input.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
config = AutoConfig.from_pretrained("google-bert/bert-base-cased")
config.num_labels = 3  # Set the number of classes
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", config=config)
def tokenize_function(dataset):
    return tokenizer(dataset["sentence"], padding="max_length", truncation=True)
tokenized_training = data_train.map(tokenize_function, batched=True)
tokenized_testing = data_val.map(tokenize_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/3876 [00:00<?, ? examples/s]

Map:   0%|          | 0/970 [00:00<?, ? examples/s]

Set up the pre-trained model.

In [ ]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    # create Trainer
training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/ipynb/final_project/BERT_financial_phrasebank_checkpoint.txt",
  learning_rate=2e-5,
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=3,
  weight_decay=0.01,
  evaluation_strategy="epoch",
  save_strategy="epoch",
  load_best_model_at_end=True,
)
def compute_metrics(eval_pred):

    f1_metric = evaluate.load("f1")

    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    results = {}
    results.update(f1_metric.compute(predictions=predictions, references = labels, average="micro"))

    return results
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=tokenized_training,
  eval_dataset=tokenized_testing,
  tokenizer=tokenizer,
  data_collator=data_collator,
  compute_metrics=compute_metrics
)


Train the model.

In [ ]:
!TORCH_USE_CUDA_DSA

/bin/bash: line 1: TORCH_USE_CUDA_DSA: command not found


In [ ]:
trainer.train()
model.save_pretrained("/content/drive/MyDrive/ipynb/final_project/BERT_financial_phrasebank")

Epoch,Training Loss,Validation Loss,F1
1,No log,0.399486,0.841237
2,No log,0.435625,0.844330
3,0.386400,0.538205,0.851546


The downstream work can start here to save time. Use the model to produce predictions

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/ipynb/final_project/BERT_financial_phrasebank")
predictions = trainer.predict(tokenized_testing)
preds = np.argmax(predictions.predictions, axis=1)


Save the labels for the downstream work.

In [ ]:
pre=preds.tolist()
with open("/content/drive/MyDrive/ipynb/final_project/BERT_financial_phrasebank.txt", 'w', encoding='utf-8') as f:
  for i in range(len(pre)):
    f.write(str(pre[i]))
    f.write('\n')


The downstream work can start here to save time. Find the total accuracy.

In [ ]:
with open("/content/drive/MyDrive/ipynb/final_project/BERT_financial_phrasebank.txt", 'r', encoding='utf-8') as f:
    result=(f.read())
result=[result]
result=result[0].split("\n")
label=[]
for i in range(len(result)-1):
  label.append(int(result[i]))
with open("/content/drive/MyDrive/ipynb/final_project/goldlabel_financial_phrasebank.txt", 'r', encoding='utf-8') as f:
    result=(f.read())
result=[result]
result=result[0].split("\n")
goldstandard=[]
for i in range(len(result)-1):
  goldstandard.append(int(result[i]))
correct=0
for i in range(len(label)):
  if label[i]==goldstandard[i]:
    correct+=1
accuracy = correct / len(label)
print("Accuracy:", accuracy)

Accuracy: 0.8412371134020619


Find the Precise.

In [ ]:
print("Macro Precision:", sklearn.metrics.precision_score(goldstandard, label, average='macro'))
print("Micro Precision:", sklearn.metrics.precision_score(goldstandard, label, average='micro'))

Macro Precision: 0.8115386765034652
Micro Precision: 0.8412371134020619


Find the Recall

In [ ]:
print("Macro Recall:", sklearn.metrics.recall_score(goldstandard, label, average='macro'))
print("Micro Recall:", sklearn.metrics.recall_score(goldstandard, label, average='micro'))

Macro Recall: 0.8328302770019516
Micro Recall: 0.8412371134020619


Find the F1

In [ ]:
print("Macro F1:", sklearn.metrics.f1_score(goldstandard, label, average='macro'))
print("Micro F1:", sklearn.metrics.f1_score(goldstandard, label, average='micro'))

Macro F1: 0.8213597041315378
Micro F1: 0.8412371134020619
